In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv
/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv
/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/dataset_dict.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/dataset_info.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/data-00000-of-00001.arrow
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/dataset_info.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/data-00000-of-00001.arrow
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/train/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/train/dataset_info.json
/kaggle/input/sam

In [2]:
!pip install -q \
  transformers==4.41.2 \
  datasets==2.19.1 \
  evaluate \
  rouge-score \
  sentencepiece


# IMPORTS

In [4]:
import pandas as pd
import re
import numpy as np
import torch

from datasets import Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)

import evaluate


2026-01-21 08:02:52.492709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768982572.516578     211 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768982572.524015     211 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768982572.542981     211 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768982572.543007     211 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768982572.543010     211 computation_placer.cc:177] computation placer alr

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


# LOAD DATASET

In [6]:
train_df = pd.read_csv(
    "/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv"
)
val_df = pd.read_csv(
    "/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv"
)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [7]:
train_df

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [8]:
train_dataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

# LIGHT TEXT CLEANING

In [10]:
def clean_text(text):
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


train_dataset = train_dataset.map(
    lambda x: {
        "dialogue": clean_text(x["dialogue"]),
        "summary": clean_text(x["summary"])
    }
)

val_dataset = val_dataset.map(
    lambda x: {
        "dialogue": clean_text(x["dialogue"]),
        "summary": clean_text(x["summary"])
    }
)



Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

# TOKENIZER

In [11]:
tokenizer = T5TokenizerFast.from_pretrained("t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

# TOKENIZATION (T5-CORRECT)

In [12]:
MAX_INPUT_LEN = 512
MAX_TARGET_LEN = 128

def preprocess_function(batch):
    inputs = ["summarize: " + d for d in batch["dialogue"]]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LEN,
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["summary"],
            max_length=MAX_TARGET_LEN,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [13]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

# DATA COLLATOR (DYNAMIC PADDING)

In [26]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model="t5-small"
)


# ROUGE METRIC

In [28]:
rouge = evaluate.load("rouge")


# ROUGE COMPUTATION FUNCTION

In [34]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Handle tuple output
    if isinstance(preds, tuple):
        preds = preds[0]

    # Convert to int64
    preds = preds.astype(np.int64)

    # 🔴 CLIP predictions to valid tokenizer range
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)

    # Replace -100 in labels and clip
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = labels.astype(np.int64)
    labels = np.clip(labels, 0, tokenizer.vocab_size - 1)

    # Decode safely
    decoded_preds = tokenizer.batch_decode(
        preds.tolist(), skip_special_tokens=True
    )
    decoded_labels = tokenizer.batch_decode(
        labels.tolist(), skip_special_tokens=True
    )

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }


# MODEL

In [30]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

# TRAINING ARGUMENTS

In [35]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_samsum",

    evaluation_strategy="epoch",
    save_strategy="epoch",

    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    num_train_epochs=10,
    weight_decay=0.01,

    save_total_limit=2,

    fp16=False,   # 🔴 IMPORTANT
    logging_steps=100,
    report_to="none",

    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=5,

    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True
)


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# SEQ2SEQ TRAINER

In [40]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)




# TRAIN

In [43]:
trainer.train()



Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,1.530300,1.718375,0.480444,0.236454,0.395888
2,1.678400,1.670835,0.485890,0.248954,0.404348
3,1.578900,1.661458,0.483604,0.247948,0.400348
4,1.492000,1.661610,0.486805,0.251114,0.403453


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3684, training_loss=1.5831041118609401, metrics={'train_runtime': 1446.2449, 'train_samples_per_second': 101.864, 'train_steps_per_second': 6.368, 'total_flos': 6026105341476864.0, 'train_loss': 1.5831041118609401, 'epoch': 4.0})

# Save the Model

In [44]:
trainer.save_model("./saved_t5_samsum")
tokenizer.save_pretrained("./saved_t5_samsum")


('./saved_t5_samsum/tokenizer_config.json',
 './saved_t5_samsum/special_tokens_map.json',
 './saved_t5_samsum/spiece.model',
 './saved_t5_samsum/added_tokens.json',
 './saved_t5_samsum/tokenizer.json')

# Reload for Inference

In [45]:
model = T5ForConditionalGeneration.from_pretrained("./saved_t5_samsum")
tokenizer = T5TokenizerFast.from_pretrained("./saved_t5_samsum")

model.to(device)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

# Inference Function (Summarization)

In [46]:
def summarize_dialogue(text):
    text = re.sub(r"\s+", " ", text).strip()
    text = "summarize: " + text

    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=128,
            num_beams=5,
            early_stopping=True
        )

    return tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )


# Test the Model

In [47]:
sample_dialogue = """
John: I might be late to the meeting.
Alice: Why?
John: Traffic is terrible.
"""

print(summarize_dialogue(sample_dialogue))


John might be late to the meeting.


In [48]:
# Test with a dialogue on a different topic
sample_dialogue = """
John: Hey Sarah, have you seen the latest tech gadget reviews? I found this new smartwatch that's supposed to have amazing health tracking features.
John: It tracks heart rate, blood oxygen levels, sleep patterns, and even stress levels! It sounds like something right up your alley. 
Sarah: That sounds really interesting! But I’ve been trying to cut down on tech distractions. I’ve heard these devices can be really overwhelming sometimes.
Sarah: I do think it’s cool that they can track so many health metrics though. I’m curious how accurate they really are.
John: Yeah, me too! There are also some new smartphones coming out with even better cameras and longer battery life. The new flagship model from XYZ brand has some insane specs.
Sarah: Ooh, I haven’t kept up with phones recently, but I’ve heard the camera quality is getting ridiculously good. It’s almost like a professional camera in your pocket now!
Sarah: Still, I feel like I’m fine with my current phone for now. I don’t really feel the need to upgrade unless something really groundbreaking comes out.
John: Totally understand that. It’s the same with me. But I think the battery life improvements are enough to make me consider it. I hate running out of battery when I’m out and about.
Sarah: That’s fair! I’m always worried about battery life too. Honestly, I think phones should last at least two full days on a single charge by now.
John: I agree! It’s so annoying when your phone dies in the middle of the day. I wonder if we’ll ever get to a point where we don’t have to charge our phones every day.
Sarah: That would be amazing! I think as tech improves, battery tech might also catch up. Let’s hope the next generation of phones can last longer!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: John found a new smartwatch with health tracking features. It tracks heart rate, blood oxygen levels, sleep patterns, and even stress levels. The new flagship model from XYZ has insane specs. Sarah is worried about battery life.


In [49]:
import os
print(os.listdir("."))


['saved_t5_samsum', '.virtual_documents', 't5_samsum']


# Model Download for building UI

In [50]:
import shutil

model_dir = "./saved_t5_samsum"
output_zip_path = "saved_summary_model.zip"

shutil.make_archive(
    base_name="saved_summary_model",
    format="zip",
    root_dir=model_dir
)

'/kaggle/working/saved_summary_model.zip'

In [51]:
from IPython.display import FileLink

# Display a download link
FileLink(output_zip_path)

/kaggle/working/saved_summary_model.zip